In [1]:
import eyenes
from eyenes import get_mario_vision_model

import numpy as np

Using TensorFlow backend.


In [2]:
from nes_py.wrappers import BinarySpaceToDiscreteSpaceEnv
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = BinarySpaceToDiscreteSpaceEnv(env, SIMPLE_MOVEMENT)

frames = []
ae_frames = []
done = True
for step in range(1000):
    if done:
        state = env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    frames.append(np.array(state))
    #env.render()
    
env.close()

frames = np.array(frames)

In [3]:
encoder, autoencoder = get_mario_vision_model()

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT, RIGHT_ONLY
from gym import wrappers
from nes_py.wrappers import BinarySpaceToDiscreteSpaceEnv


import keras
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, Dense, Flatten, MaxPooling1D, SeparableConv2D, Activation
from keras.layers import AveragePooling2D, MaxPooling2D, LSTM, Concatenate, Reshape, GRU, BatchNormalization
from keras.initializers import Constant
from keras.constraints import MaxNorm
from keras.applications.xception import Xception
from keras.applications.mobilenet_v2 import MobileNetV2

import keras
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, Conv3D, Dense, Flatten, MaxPooling1D, SeparableConv2D, Activation, Lambda
from keras.layers import AveragePooling2D, MaxPooling2D, LSTM, Concatenate, Reshape, GRU, BatchNormalization, UpSampling2D
from keras.layers import Cropping2D, ZeroPadding2D
from keras.initializers import Constant
from keras.constraints import MaxNorm
from keras.applications.xception import Xception
from keras.applications.mobilenet_v2 import MobileNetV2

import tensorflow as tf

def make_env(version, movement_type):
    env = gym_super_mario_bros.make('SuperMarioBros-v' + str(version))
    env = BinarySpaceToDiscreteSpaceEnv(env, movement_type)
    return env

version = 3
movement_type = SIMPLE_MOVEMENT
env = make_env(version, movement_type)
obs_shape = env.observation_space.shape
square_shape = (16,16)
strides = int(square_shape[0]/2)
output_dim = len(env.get_action_meanings())


def get_mario_vision_model(obs_shape = obs_shape, square_shape = square_shape, strides = strides, output_dim = output_dim, hidden_size = 10):
    image = Input((240, 256, 3))
    encoded = ZeroPadding2D(padding = (8,0))(image)
    encoded = Lambda(function = lambda x: x/255.0)(encoded)
    encoded = Conv2D(kernel_size = (8,8), filters=3, padding = 'same', activation = 'relu')(encoded)
    encoded = MaxPooling2D((4,4))(encoded)
    encoded = Conv2D(kernel_size = (4,4), filters=6, padding = 'same', activation = 'relu')(encoded)
    encoded = MaxPooling2D((4,4))(encoded)
    encoded = Conv2D(kernel_size = (2,2), filters=12, padding = 'same', activation = 'relu')(encoded)
    encoded = MaxPooling2D((2,2))(encoded)
    encoded = Conv2D(kernel_size = (2,2), filters=24, padding = 'same', activation = 'relu')(encoded)
    encoded = MaxPooling2D((2,2))(encoded)
    
    encoder = Model(image, encoded)
 
    decoded = UpSampling2D((2,2))(encoded)
    decoded = Conv2D(kernel_size = (2,2), filters=12, activation = 'relu', padding = 'same')(decoded)
    decoded = UpSampling2D((2,2))(decoded)
    decoded = Conv2D(kernel_size = (2,2), filters=6, activation = 'relu', padding = 'same')(decoded)
    decoded = UpSampling2D((4,4))(decoded)
    decoded = Conv2D(kernel_size = (4,4), filters=3, activation = 'relu', padding = 'same')(decoded)
    decoded = UpSampling2D((4,4))(decoded)
    decoded = Conv2D(kernel_size = (8,8), filters=3, activation = 'relu', padding = 'same')(decoded)
    decoded = Cropping2D((8,0))(decoded)
    autoencoder = Model(image, decoded)
    
    autoencoder.compile(optimizer = 'adam', loss = 'mse')
    return encoder, autoencoder

In [ ]:
encoder, autoencoder = get_mario_vision_model()

In [ ]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 240, 256, 3)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 256, 256, 3)       0         
_________________________________________________________________
lambda_4 (Lambda)            (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 256, 256, 3)       579       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 64, 6)         294       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 6)         0         
__________

In [ ]:
autoencoder.fit(frames,frames, epochs = 100, batch_size = 200, verbose = 2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100


In [ ]:
res = autoencoder.predict(frames, batch_size = 1)

In [ ]:
plt.imshow(res[0])
plt.show()

In [ ]:
np.shape(res)